<a href="https://colab.research.google.com/github/SBShimm/Aiffel/blob/master/exploration/Exploration13_SB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Movielens 영화 추천

먼저 노드에서 진행했던 방식과 유사하게 진행하기 위해 implicit의 버전을 다운그레이드 해줍니다.  
코랩 버전인 0.7.1로 진행할 경우 함수의 출력그거 벡터의 형상이 다르게 나옵니다.

In [ ]:
!pip install --upgrade pip setuptools wheel
!pip install opencv-python

In [ ]:
! pip install --upgrade implicit=='0.4.8'

In [3]:
import numpy as np
import scipy
import implicit
import pandas as pd
import numpy as np

print(np.__version__)
print(scipy.__version__)
print(implicit.__version__)

1.21.6
1.7.3
0.4.8


## 1. 데이터 전처리  
먼저, 데이터를 불러옵니다. 데이터는 movielens의 평점데이터 입니다. 

In [4]:
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv('/content/drive/MyDrive/Colab/Datasets/ml-1m/ratings.dat', sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


이 데이터는 이미 user_id, movie_id로 indexing이 되어 있습니다.  
여기서 영화 추천에 3점 미만의 데이터는 사용하지 않겠습니다.

In [5]:
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
ratings.rename(columns={'ratings':'counts'}, inplace=True)

In [7]:
ratings['counts']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: counts, Length: 836478, dtype: int64

이미 인덱싱 되어있는 데이터의 영화 제목을 알기위해 id와 title정보가 있는 데이터를 추가적으로 사용하도록 하겠습니다.

In [8]:
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv('/content/drive/MyDrive/Colab/Datasets/ml-1m/movies.dat', sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


id를 기준으로 조인해 줍시다.

In [9]:
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,counts,timestamp,title
0,1,1193,5,978300760,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,978302109,James and the Giant Peach (1996)
2,1,914,3,978301968,My Fair Lady (1964)
3,1,3408,4,978300275,Erin Brockovich (2000)
4,1,2355,5,978824291,"Bug's Life, A (1998)"
...,...,...,...,...,...
836473,6040,1090,3,956715518,Platoon (1986)
836474,6040,1094,5,956704887,"Crying Game, The (1992)"
836475,6040,562,5,956704746,Welcome to the Dollhouse (1995)
836476,6040,1096,4,956715648,Sophie's Choice (1982)


이제 각각 영화의 수, 사용자 수, 인기가 많은 영화 30개를 한번 확인해 보겠습니다.


In [10]:
# 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [11]:
# 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [12]:
# 인기있는 영화 30개
movie_count = ratings.groupby('title')['counts'].count()
movie_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

익숙한 영화들이 많네요.  
여기서 제가 좋아하는 영화를 데이터로 추가하겠습니다.

In [13]:
my_favorite = ['Fargo (1996)', 'Toy Story (1995)', 'Men in Black (1997)' , 'Back to the Future (1985)', 'Saving Private Ryan (1998)']

new_lines = pd.DataFrame({'user_id':['SBShim']*5, 'title':my_favorite, 'counts':[5]*5})

ratings = ratings.append(new_lines)
ratings.tail(10)

,user_id,movie_id,counts,timestamp,title
836473,6040,1090.0,3,956715518.0,Platoon (1986)
836474,6040,1094.0,5,956704887.0,"Crying Game, The (1992)"
836475,6040,562.0,5,956704746.0,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,956715648.0,Sophie's Choice (1982)
836477,6040,1097.0,4,956715569.0,E.T. the Extra-Terrestrial (1982)
0,SBShim,NaN,5,NaN,Fargo (1996)
1,SBShim,NaN,5,NaN,Toy Story (1995)
2,SBShim,NaN,5,NaN,Men in Black (1997)
3,SBShim,NaN,5,NaN,Back to the Future (1985)
4,SBShim,NaN,5,NaN,Saving Private Ryan (1998)


정상적으로 추가됐네요.  
아까 확인했을 때, user_id는 6040까지 있는데 unique가 6039개, movie_id는 3800번대 까지 있는데 unique는 3600대의 수를 보였습니다. 중간중간에 빈 자리가 있는것 같고 SBShim으로 데이터도 추가해 줬으니 다시 indexing 해주도록 하겠습니다.

In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['title'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [15]:
# 인덱싱이 잘 되었는지 확인해 봅니다. 
print(user_to_idx['SBShim'])
print(movie_to_idx['Fargo (1996)'])

6039
51


In [16]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

user_id column indexing OK!!
movie_id column indexing OK!!


In [17]:
ratings

,user_id,movie_id,counts,timestamp,title
0,0,0,5,978300760.0,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,978302109.0,James and the Giant Peach (1996)
2,0,2,3,978301968.0,My Fair Lady (1964)
3,0,3,4,978300275.0,Erin Brockovich (2000)
4,0,4,5,978824291.0,"Bug's Life, A (1998)"
...,...,...,...,...,...
0,6039,51,5,NaN,Fargo (1996)
1,6039,40,5,NaN,Toy Story (1995)
2,6039,175,5,NaN,Men in Black (1997)
3,6039,22,5,NaN,Back to the Future (1985)


인덱싱이 끝난 데이터를 이용하여 CSR Matrix를 생성하겠습니다.

In [18]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movies = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['counts'], (ratings.user_id, ratings.movie_id)), shape=(num_user, num_movies))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 2. 모델 구성하기 
이제 학습할 모델을 준비합니다.  
implicit 패키지의 als 모델을 사용합니다. 

In [19]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [20]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)

In [21]:
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

준비한 모델로 학습을 시켜줍시다.

In [22]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

## 3. 영화 추천
이제 각각 제가 추가한 평점 데이터를 이용하여 제가 평점을 남겼던 영화와 평점을 남기지 않은 영화를 통해 선호도를 체크해 보겠습니다.

In [23]:
user, movie1, movie2 = user_to_idx['SBShim'], movie_to_idx['James and the Giant Peach (1996)'], movie_to_idx['Toy Story (1995)']
user_vector, movie1_vector, movie2_vector = als_model.user_factors[user], als_model.item_factors[movie1], als_model.item_factors[movie2]

In [24]:
user_vector

array([ 0.79728806,  0.36559203, -0.4981043 , -1.0443616 ,  0.02600287,
        0.21039541,  0.03202421,  0.1160691 ,  0.9084425 , -0.70759195,
       -0.21348672, -0.12428351,  0.86088455, -0.81466836,  0.75681955,
        0.7916172 , -0.3388649 , -0.28935468, -0.4255776 ,  0.14504634,
       -0.70375764, -0.20063971, -0.7503045 ,  0.43174228, -0.07368468,
       -0.5113064 ,  0.04554807,  0.42721617,  0.05908995, -0.10360583,
       -0.5072723 , -0.35585484,  0.2979387 , -0.38709664, -0.41663483,
        0.49788514,  0.10685575,  0.3770724 , -0.17657438, -0.09477576,
        0.22077128,  0.9894096 ,  1.1284789 ,  0.42195943,  0.15777439,
       -0.45691934, -0.5191613 , -0.21372359,  0.43190548, -0.08012772,
       -0.3175971 , -0.0510447 ,  0.2515708 ,  0.12453848, -0.30559048,
        0.50535876,  0.1715025 ,  0.01735882, -0.23189071,  0.52392614,
       -0.5293061 ,  0.28440052, -0.6221826 ,  0.22442389,  0.28741992,
       -0.93437535, -0.36910188, -0.12602463,  0.15376167,  0.75

각각 user, 영화에 대한 vector를 model에서 받아와 내적 연산을 하게되면 선호도를 얻을 수 있습니다.

In [25]:
np.dot(user_vector, movie1_vector)

0.048869222

In [26]:
np.dot(user_vector, movie2_vector)

0.5072734

제가 5점으로 남겼던 작품이 더 높게 나오네요!  
다음은 유사한 영화 찾기입니다.  
Back to the Future와 유사한 영화를 찾아보겠습니다.

In [27]:
favorite_movie = 'Back to the Future (1985)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(22, 0.9999999),
 (243, 0.55389404),
 (13, 0.5444437),
 (674, 0.5322554),
 (488, 0.5043911),
 (19, 0.49264202),
 (615, 0.47705856),
 (1144, 0.45814246),
 (169, 0.4434717),
 (117, 0.4358669),
 (163, 0.43477678),
 (26, 0.42214692),
 (3534, 0.4206861),
 (15, 0.41112664),
 (44, 0.40169498)]

이렇게 결과가 나오네요 이렇게만 봐서는 무슨 영화인지 알 수가 없으니 제목으로 변환하겠습니다.

In [28]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Back to the Future (1985)',
 'Ghostbusters (1984)',
 "Ferris Bueller's Day Off (1986)",
 'Back to the Future Part II (1989)',
 'When Harry Met Sally... (1989)',
 'Big (1988)',
 'Cocoon (1985)',
 'Bull Durham (1988)',
 'Fish Called Wanda, A (1988)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Raising Arizona (1987)',
 'E.T. the Extra-Terrestrial (1982)',
 'Project Moon Base (1953)',
 'Airplane! (1980)',
 'Star Wars: Episode IV - A New Hope (1977)']

자신을 포함해 2편 영화도 나오고 대략 선호도가 비슷한 영화들이 이렇게 나옵니다.  
다음은 user가 남긴 평점을 기준으로 다른 영화를 추천해 보도록 합시다.

In [29]:
user = user_to_idx['SBShim']
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(23, 0.5166955),
 (87, 0.49413452),
 (157, 0.4699536),
 (50, 0.46482927),
 (121, 0.4422328),
 (107, 0.4214314),
 (92, 0.40188712),
 (99, 0.371269),
 (160, 0.34449548),
 (124, 0.32878485),
 (222, 0.3159837),
 (4, 0.30545694),
 (110, 0.2978736),
 (5, 0.25746882),
 (80, 0.2568621),
 (269, 0.25447813),
 (117, 0.25313628),
 (64, 0.24875927),
 (248, 0.24406508),
 (0, 0.24395259)]

In [30]:
[idx_to_movie[i[0]] for i in movie_recommended]

["Schindler's List (1993)",
 'Braveheart (1995)',
 'Shawshank Redemption, The (1994)',
 'Toy Story 2 (1999)',
 'Silence of the Lambs, The (1991)',
 'Jurassic Park (1993)',
 'Terminator 2: Judgment Day (1991)',
 'American Beauty (1999)',
 'Forrest Gump (1994)',
 'Matrix, The (1999)',
 'Pulp Fiction (1994)',
 "Bug's Life, A (1998)",
 'Groundhog Day (1993)',
 'Princess Bride, The (1987)',
 'Stand by Me (1986)',
 'GoodFellas (1990)',
 'Star Wars: Episode V - The Empire Strikes Back (1980)',
 'Star Wars: Episode VI - Return of the Jedi (1983)',
 'Good Will Hunting (1997)',
 "One Flew Over the Cuckoo's Nest (1975)"]

대체적으로 SF영화와 애니메이션들이 추천된 것을 볼 수 있습니다. Toy Story와 다른 SF영화들의 영향인 것 같습니다.

## 4. 회고
1. 이번 익스는 너무 어렵지는 않아서 그랬는지 더 재미있었다.
2. 시간이 나면 추천받은 영화들을 한번 보도록 해야겠다.